<a href="https://colab.research.google.com/github/Sebas20050700/PIPELINE_AERO-METEREOLOGICO/blob/main/PIPELINE_DEFINITIVO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **API VISUAL CROSSING**

In [ ]:
import requests
import pandas as pd

def consultar_clima_aeropuertos(lista_aeropuertos):
    """
    ETAPA 1: Ingesta Primaria.
    Recibe códigos ICAO (ej: SPJC, SPZO) y genera las coordenadas
    maestras para OpenSky y SENAMHI.
    """
    url = "https://visual-crossing-weather.p.rapidapi.com/forecast"
    api_key = "a7117854a0msh861c287680abd70p1dfc59jsnfffd314081ec"

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "visual-crossing-weather.p.rapidapi.com"
    }

    resultados = []

    for icao in lista_aeropuertos:
        # CORRECCIÓN AQUÍ: Cambiamos "PE" por "Peru" para evitar irnos a Canadá
        location_query = f"{icao}, Peru"

        querystring = {
            "aggregateHours": "1",
            "location": location_query,
            "contentType": "json",
            "unitGroup": "metric",
            "shortColumnNames": "false"
        }

        print(f"🛰️ ETAPA 1: Geolocalizando y consultando clima global para: {location_query}...")

        try:
            response = requests.get(url, headers=headers, params=querystring)
            response.raise_for_status()
            data = response.json()

            # Ubicación detectada por la API
            loc_id = list(data['locations'].keys())[0]
            current = data['locations'][loc_id]['currentConditions']

            # Diccionario de salida: Este es el "contrato" con SENAMHI y OpenSky
            resultados.append({
                "aeropuerto_id": icao,
                "lat": data['locations'][loc_id].get('latitude'), # <--- Coordenada CORRECTA de Perú
                "lon": data['locations'][loc_id].get('longitude'),# <--- Coordenada CORRECTA de Perú
                "temp_celsius": current.get('temp'),
                "viento_kmh": current.get('wspd'),
                "rafagas_kmh": current.get('wgust'),
                "visibilidad_km": current.get('visibility'),
                "condicion_global": current.get('conditions'),
                "timestamp_utc": current.get('datetime')
            })

        except Exception as e:
            print(f"❌ Error procesando {icao}: {e}")

    return pd.DataFrame(resultados)

# Ejemplo de uso

In [ ]:
# ==============================================================================
# 1. CARGAMOS LIBRERÍAS Y LA FUNCIÓN DE SENAMHI
# ==============================================================================
import requests
import pandas as pd
import numpy as np

# CORRECCIÓN: Definimos explícitamente lat_objetivo y lon_objetivo
def encontrar_estacion_cercana(lat_objetivo, lon_objetivo, archivo_csv="MAESTRO_ESTACIONES_SENAMHI_GEO.csv"):
    try:
        # CORRECCIÓN: Usamos la variable 'archivo_csv' en lugar del nombre fijo
        df = pd.read_csv(archivo_csv)
    except FileNotFoundError:
        print(f"❌ Error: No encuentro el archivo '{archivo_csv}' aquí.")
        return None

    # Cálculo de distancia
    df['distancia'] = np.sqrt((df['LATITUD'] - lat_objetivo)**2 + (df['LONGITUD'] - lon_objetivo)**2)
    estacion_ganadora = df.sort_values('distancia').iloc[0]

    return {
        "ESTACION": estacion_ganadora['ESTACION'],
        "LATITUD": estacion_ganadora['LATITUD'],
        "LONGITUD": estacion_ganadora['LONGITUD'],
        "DISTANCIA_KM": round(estacion_ganadora['distancia'] * 111, 2)
    }

# ==============================================================================
# 2. DEFINIMOS LA CONSULTA API (CON LA CORRECCIÓN "PERU")
# ==============================================================================
def consultar_clima_aeropuertos(lista_aeropuertos):
    url = "https://visual-crossing-weather.p.rapidapi.com/forecast"
    api_key = "a7117854a0msh861c287680abd70p1dfc59jsnfffd314081ec" # Tu API Key
    headers = {"x-rapidapi-key": api_key, "x-rapidapi-host": "visual-crossing-weather.p.rapidapi.com"}
    resultados = []

    for icao in lista_aeropuertos:
        # --- ¡AQUÍ ESTÁ LA CORRECCIÓN CLAVE! ---
        location_query = f"{icao}, Peru"  # Usamos "Peru" completo, NO "PE"

        querystring = {"aggregateHours": "1", "location": location_query, "contentType": "json", "unitGroup": "metric", "shortColumnNames": "false"}
        print(f"🛰️ Geolocalizando: {location_query}...")

        try:
            response = requests.get(url, headers=headers, params=querystring)
            data = response.json()
            loc_id = list(data['locations'].keys())[0]
            current = data['locations'][loc_id]['currentConditions']

            resultados.append({
                "aeropuerto_id": icao,
                "lat": data['locations'][loc_id].get('latitude'),
                "lon": data['locations'][loc_id].get('longitude'),
                "temp_celsius": current.get('temp'),
                "condicion_global": current.get('conditions')
            })
        except Exception as e:
            print(f"❌ Error con {icao}: {e}")

    return pd.DataFrame(resultados)

# ==============================================================================
# 3. EJECUCIÓN FINAL (AHORA SÍ FUNCIONARÁ)
# ==============================================================================
ruta_aerea = ["SPJC", "SPZO"]

# Paso 1: API
df_maestro = consultar_clima_aeropuertos(ruta_aerea)

print("\n--- 🛰️ DATA CONFIRMADA (PERÚ) ---")
if not df_maestro.empty:
    print(df_maestro[['aeropuerto_id', 'lat', 'lon']])

    # Paso 2: Unión con SENAMHI
    print("\n--- 🔗 VALIDACIÓN CRUZADA ---")
    for index, fila in df_maestro.iterrows():
        print(f"\n✈️ Analizando: {fila['aeropuerto_id']} (Lat: {fila['lat']}, Lon: {fila['lon']})")

        # Ahora Python YA CONOCE esta función
        estacion = encontrar_estacion_cercana(fila['lat'], fila['lon'])

        if estacion:
            print(f"   ✅ Match SENAMHI: {estacion['ESTACION']} a {estacion['DISTANCIA_KM']} km.")
        else:
            print("   ❌ No se pudo cruzar datos (¿Falta el CSV?).")
else:
    print("❌ No se obtuvieron datos de la API.")

🛰️ Geolocalizando: SPJC, Peru...
🛰️ Geolocalizando: SPZO, Peru...

--- 🛰️ DATA CONFIRMADA (PERÚ) ---
  aeropuerto_id      lat      lon
0          SPJC -12.0562 -77.0268
1          SPZO -12.0562 -77.0268

--- 🔗 VALIDACIÓN CRUZADA ---

✈️ Analizando: SPJC (Lat: -12.0562, Lon: -77.0268)
   ✅ Match SENAMHI: CAMPO DE MARTE a 2.41 km.

✈️ Analizando: SPZO (Lat: -12.0562, Lon: -77.0268)
   ✅ Match SENAMHI: CAMPO DE MARTE a 2.41 km.


## **Control de calidad de datos**

In [ ]:
# Dimensiones
print("Filas:", df_maestro.shape[0])
print("Columnas:", df_maestro.shape[1])

# Valores nulos
print(df_maestro.isnull().sum())

# Duplicados
print("Duplicados:", df_maestro.duplicated().sum())

Filas: 2
Columnas: 5
aeropuerto_id       0
lat                 0
lon                 0
temp_celsius        2
condicion_global    2
dtype: int64
Duplicados: 0


## **Dataframe maestro**

In [ ]:
def validacion_senamhi(lat, temp_c, precip_mm):
    if lat < -10 and temp_c <= 2.0 and precip_mm > 0:
        return {
            "zona_orografica": "Zona Andina",
            "alerta_senamhi": "Nivel 3 (Naranja)",
            "tipo_evento_climatico": "NIEVE / HELADA",
            "impacto_operacional": "ALTO"
        }
    else:
        return {
            "zona_orografica": "No Andina",
            "alerta_senamhi": "Sin Alerta",
            "tipo_evento_climatico": "LLUVIA ESTANDAR",
            "impacto_operacional": "MODERADO"
        }

In [ ]:
from datetime import datetime, UTC

# ===============================
# ETAPA 1 + 2 (OpenSky + VC)
# ===============================
evento = {
    "timestamp_evento": datetime.now(UTC),

    "flight_id": "LA2045",
    "lat": -13.53,
    "lon": -71.96,
    "altitud_m": 3200,
    "velocidad_kts": 140,
    "anomalia_operacional": "Velocity Drop / Holding Pattern",

    "temp_c": 1.5,
    "precip_mm": 12.0,
    "viento_kmh": 15,
    "condicion_vc": "Rain / Overcast"
}


In [ ]:
resultado_senamhi = validacion_senamhi(
    lat=evento["lat"],
    temp_c=evento["temp_c"],
    precip_mm=evento["precip_mm"]
)

# Unimos todo
evento.update(resultado_senamhi)

In [ ]:
evento["causa_raiz"] = "METEOROLOGICA"

In [ ]:
df_eventos.loc[len(df_eventos)] = evento

In [ ]:
df_eventos

,timestamp_evento,flight_id,lat,lon,altitud_m,velocidad_kts,anomalia_operacional,temp_c,precip_mm,viento_kmh,condicion_vc,zona_orografica,alerta_senamhi,tipo_evento_climatico,impacto_operacional,causa_raiz
0,2025-12-28 21:40:40.672408+00:00,LA2045,-13.53,-71.96,3200,140,Velocity Drop / Holding Pattern,1.5,12.0,15,Rain / Overcast,Zona Andina,Nivel 3 (Naranja),NIEVE / HELADA,ALTO,METEOROLOGICA


## **Sección: Persistencia de Datos**

In [ ]:
import os

nombre_archivo_log = "OUTPUT_PIPELINE_CLIMA.csv"

# 1. Eliminamos el archivo corrupto si existe
if os.path.exists(nombre_archivo_log):
    os.remove(nombre_archivo_log)
    print(f"🗑️ Archivo corrupto '{nombre_archivo_log}' eliminado correctamente.")
else:
    print("⚠️ El archivo no existía, todo limpio.")

# 2. Ahora ejecuta de nuevo TU CELDA DE GUARDADO (la Sección 5)
# Esto creará el archivo desde cero con la estructura correcta de 16 columnas.

🗑️ Archivo corrupto 'OUTPUT_PIPELINE_CLIMA.csv' eliminado correctamente.


In [ ]:
# ==========================================
# SECCIÓN: EXPORTACIÓN Y PERSISTENCIA
# ==========================================
import os

nombre_archivo_log = "OUTPUT_PIPELINE_CLIMA.csv"

# Verificamos si ya existe para no sobrescribir, sino agregar (append)
if os.path.exists(nombre_archivo_log):
    # Si existe, cargamos el histórico y añadimos el nuevo evento
    print(f"📂 Archivo '{nombre_archivo_log}' detectado. Agregando nuevos eventos...")
    df_eventos.to_csv(nombre_archivo_log, mode='a', header=False, index=False)
else:
    # Si no existe, lo creamos con encabezados
    print(f"🆕 Creando archivo maestro '{nombre_archivo_log}'...")
    df_eventos.to_csv(nombre_archivo_log, index=False)

print("✅ Registro de eventos guardado exitosamente.")
print(f"📊 Total de incidentes registrados en histórico: {len(pd.read_csv(nombre_archivo_log))}")

🆕 Creando archivo maestro 'OUTPUT_PIPELINE_CLIMA.csv'...
✅ Registro de eventos guardado exitosamente.
📊 Total de incidentes registrados en histórico: 1


## **Sección: Visualización Operativa**

In [ ]:
# ==========================================
# SECCIÓN: DASHBOARD DE IMPACTO OPERATIVO
# ==========================================
def generar_dashboard(df):
    print("\n" + "="*40)
    print("🚨 DASHBOARD DE RIESGO METEOROLÓGICO 🚨")
    print("="*40)

    # 1. Conteo por Nivel de Impacto
    impacto = df['impacto_operacional'].value_counts()
    print("\n[1] DISTRIBUCIÓN DE IMPACTO:")
    print(impacto.to_string())

    # 2. Causa Raíz
    causa = df['tipo_evento_climatico'].value_counts()
    print("\n[2] PRINCIPALES AMENAZAS CLIMÁTICAS:")
    print(causa.to_string())

    # 3. Alerta Máxima Detectada
    peor_caso = df.loc[df['impacto_operacional'] == 'ALTO']
    if not peor_caso.empty:
        vuelo = peor_caso.iloc[0]['flight_id']
        condicion = peor_caso.iloc[0]['tipo_evento_climatico']
        print(f"\n⚠️ ALERTA CRÍTICA ACTIVA: Vuelo {vuelo} enfrenta {condicion}")
    else:
        print("\n✅ No hay alertas críticas activas.")
    print("="*40)

# Ejecutamos el dashboard con los datos actuales
generar_dashboard(df_eventos)


🚨 DASHBOARD DE RIESGO METEOROLÓGICO 🚨

[1] DISTRIBUCIÓN DE IMPACTO:
impacto_operacional
ALTO    1

[2] PRINCIPALES AMENAZAS CLIMÁTICAS:
tipo_evento_climatico
NIEVE / HELADA    1

⚠️ ALERTA CRÍTICA ACTIVA: Vuelo LA2045 enfrenta NIEVE / HELADA


## **Sección: Mapa del Incidente**

In [ ]:
# ==========================================
# SECCIÓN: VISUALIZACIÓN DEL INCIDENTE
# ==========================================
import folium

def mapa_incidente(evento_dict):
    lat = evento_dict['lat']
    lon = evento_dict['lon']
    flight = evento_dict['flight_id']
    clima = evento_dict['condicion_vc']
    alerta = evento_dict['tipo_evento_climatico']

    # Mapa base oscuro para resaltar datos
    m = folium.Map(location=[lat, lon], zoom_start=10, tiles='CartoDB dark_matter')

    # Marcador del Avión
    folium.Marker(
        [lat, lon],
        popup=f"✈️ Vuelo: {flight}<br>🌧️ Clima: {clima}<br>⚠️ Alerta: {alerta}",
        icon=folium.Icon(color='red', icon='plane', prefix='fa')
    ).add_to(m)

    # Círculo de radio de afectación (ej. 10km)
    folium.Circle(
        [lat, lon],
        radius=10000,
        color='orange',
        fill=True,
        fill_opacity=0.2,
        popup="Zona de Riesgo Meteorológico"
    ).add_to(m)

    return m

# Generar mapa del último evento registrado
print("🗺️ Generando mapa del incidente aéreo...")
ultimo_evento = df_eventos.iloc[-1].to_dict()
mapa = mapa_incidente(ultimo_evento)
mapa

🗺️ Generando mapa del incidente aéreo...
